## Use opence-v1.4.1

In [14]:
#  # mount drive
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
# root_dir = "/content/gdrive/MyDrive"
# # change the OS to use your project folder as the working directory
# notebooks = "/post_ocr_repository/post_ocr_correction/notebooks/en"
# import os
# os.chdir(root_dir + notebooks)

In [15]:
# # HAL
# file_dir = '/home/jnaiman/wwt_image_extraction/alignments/'
# output_folder = '/home/jnaiman/data/morgan/'
# # utils from local
# from sys import path
# path.append('/home/jnaiman/libraries/')
# from utils_ocr_mini import align_texts_fast

# local
file_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/alignments/'
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/'
from sys import path
path.append('../scienceDigitization/text_mining_ocr_and_pdf/ocr_spell_check/')
from importlib import reload
import utils
reload(utils)
from utils import align_texts_fast, get_fill_in_types

realign_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/ocr_pdf_aligned_sents_each_realign5/'


ender = '_small_words_pageLevel'


window_length = 100 # for subsetting the data

npages_train = 1000
npages_val = 250
npages_test = 250

In [16]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import numpy as np

#from sys import path
#path.append('/home/jnaiman/.local')

from nltk.lm import Vocabulary
#import sys
#sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from glob import glob

from Levenshtein import distance
import re
from tqdm.auto import tqdm
import Levenshtein

import time
#from multiprocessing import Pool
from multiprocess import Pool # not sure if this is needed only on the mac?


# from importlib import reload
# import utils_ocr_mini
# reload(utils_ocr_mini)
# from utils_ocr_mini import align_texts_fast

In [17]:
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]

def levenshtein(reference, hypothesis, progress_bar = False):
    assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

Get all data, format maybe:

In [18]:
align_files = glob(realign_dir + '*.pickle')
len(align_files)

7857

Load training data:

In [19]:
def make_dataframe(npages, arts_save = [], pages_save = [], imod=1000):
    # arts_save = []
    # pages_save = []
    train_pdf = []; train_ocr = []
    pages_save_out = []; filenames = []
    pdf_clean = []; ocr_clean = []
    while len(train_pdf) < npages:
        if len(train_pdf)%imod == 0:
            print('on', len(train_pdf), 'of', npages)
        i = np.random.randint(len(align_files))
        with open(align_files[i],'rb') as f:
            article = pickle.load(f)
        page_keys = list(article.keys())
        if len(page_keys) > 0:
            j = np.random.randint(len(page_keys))
            # check already in there
            alreadyIn = False
            if align_files[i].split('/')[-1].split('.pickle')[0] in arts_save:
                ind = arts_save.index(align_files[i].split('/')[-1].split('.pickle')[0])
                if page_keys[j] in pages_save: # also in there, correct one?
                    if pages_save[ind] == page_keys[j]: # same index
                        alreadyIn = True
            if not alreadyIn:
                arts_save.append(align_files[i].split('/')[-1].split('.pickle')[0])
                pages_save.append(page_keys[j])
                if article[page_keys[j]] != {}:
                    train_pdf.append(article[page_keys[j]]['PDF'])
                    # for formatting
                    train_ocr.append(article[page_keys[j]]['OCR'].replace('^','@'))
                    # also cleaned
                    pdf_clean.append(article[page_keys[j]]['PDF'].replace('@',''))
                    ocr_clean.append(article[page_keys[j]]['OCR'].replace('^',''))
                    pages_save_out.append(page_keys[j])
                    filenames.append(align_files[i].split('/')[-1].split('.pickle')[0])

    train_df = pd.DataFrame({'gt':train_pdf, 'ocr':train_ocr, 'filename':filenames, 
                             'page':pages_save_out,
                            'gt_clean':pdf_clean, 'ocr_clean':ocr_clean})
    return train_df, arts_save, pages_save

In [20]:
train_data, arts_save, pages_save = make_dataframe(npages_train,
                                                   arts_save = [], pages_save = [])

on 0 of 1000


In [21]:
val_data, arts_save, pages_save = make_dataframe(npages_val, 
                                                arts_save=arts_save.copy(),
                                               pages_save=pages_save.copy())

on 0 of 250
on 0 of 250


In [22]:
test_data, arts_save, pages_save = make_dataframe(npages_test, 
                                                arts_save=arts_save.copy(),
                                               pages_save=pages_save.copy())

on 0 of 250
on 0 of 250
on 0 of 250
on 0 of 250
on 0 of 250


Combine all together to for vocabulary:

In [23]:
data = pd.concat([train_data,val_data,test_data],ignore_index=True)

In [24]:
data.head()

,gt,ocr,filename,page,gt_clean,ocr_clean
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,1111_1111.5432d_hatp13_psfilefixedRTM_hocr,page8,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,". Finally, the accurate, observational determi...",@ Finally. the accurate. observational determi...,1009_1009.1978d_ms_psfilefixedRTM_hocr,page1,". Finally, the accurate, observational determi...",Finally. the accurate. observational determin...
2,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...,1007_1007.0493d_ms_psfilefixedRTM_hocr,page1,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...
3,where with are the density fractions of each m...,where with are the deusity fractions of each m...,0810_0810.0555d_11077aph_psfilefixedRTM_hocr,page4,where with are the density fractions of each m...,where with are the deusity fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,1106_1106.1432d_ms_psfilefixedRTM_hocr,page1,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


In [25]:
# take out any weirdos
mask = (pd.isnull(data['gt'])) | (pd.isnull(data['ocr']))

In [26]:
len(data[mask])

0

In [27]:
data = data[~mask]

In [28]:
data.head()

,gt,ocr,filename,page,gt_clean,ocr_clean
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,1111_1111.5432d_hatp13_psfilefixedRTM_hocr,page8,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,". Finally, the accurate, observational determi...",@ Finally. the accurate. observational determi...,1009_1009.1978d_ms_psfilefixedRTM_hocr,page1,". Finally, the accurate, observational determi...",Finally. the accurate. observational determin...
2,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...,1007_1007.0493d_ms_psfilefixedRTM_hocr,page1,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...
3,where with are the density fractions of each m...,where with are the deusity fractions of each m...,0810_0810.0555d_11077aph_psfilefixedRTM_hocr,page4,where with are the density fractions of each m...,where with are the deusity fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,1106_1106.1432d_ms_psfilefixedRTM_hocr,page1,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


Rename with equivalents for the original OCR method:

In [29]:
# train = train.rename(columns={"sentences source": "ocr_aligned", 
#                         "sentences target": "gs_aligned"})

data = data.rename(columns={'ocr_clean':"ocr_to_input", 
                            'ocr': "ocr_aligned",
                            'gt':"gs_aligned"})

In [30]:
data[['ocr_to_input','ocr_aligned', 'gs_aligned']].head()

,ocr_to_input,ocr_aligned,gs_aligned
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,Finally. the accurate. observational determin...,@ Finally. the accurate. observational determi...,". Finally, the accurate, observational determi..."
2,galaxies. BECs are known to have positive colo...,galaxies. BECs are known to have positive colo...,galaxies. BEGs are known to have positive colo...
3,where with are the deusity fractions of each m...,where with are the deusity fractions of each m...,where with are the density fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


In [31]:
data[['ocr_to_input','ocr_aligned', 'gs_aligned']].applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,1500.000000,1500.000000,1500.000000
mean,3014.155333,3041.941333,3041.941333
std,1585.452724,1600.632865,1600.632865
min,1.000000,1.000000,1.000000
25%,1872.250000,1892.750000,1892.750000
50%,2866.500000,2897.500000,2897.500000
75%,4131.250000,4177.500000,4177.500000
max,7114.000000,8625.000000,8625.000000


In [32]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count    1500.000000
mean        5.413870
std         6.446605
min         0.000000
25%         2.402198
50%         4.319378
75%         6.750639
max        88.208617
Name: cer, dtype: float64

Create vocabulary:

In [33]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    1500.000000
mean        5.235680
std         5.824194
min         0.000000
25%         2.370930
50%         4.249685
75%         6.629447
max        83.165217
Name: cer, dtype: float64

In [34]:
# let's try with just test dataset:
#data = pd.concat([train_data,val_data,test_data],ignore_index=True)
levenshtein(reference = data.iloc[-len(test_data):].gs_aligned.replace("@",""), 
            hypothesis = data.iloc[-len(test_data):].ocr_aligned.replace("@","")).cer.describe()

count    250.000000
mean       6.048617
std        8.439131
min        0.000000
25%        2.599628
50%        4.564981
75%        7.002886
max       83.165217
Name: cer, dtype: float64

In [35]:
test_data

,gt,ocr,filename,page,gt_clean,ocr_clean
0,Define A@:= Since a complement space to the im...,Define AA:= Since a complement space to the im...,1112_1112.2544d_1112.2544_psfilefixedRTM_hocr,page6,Define A:= Since a complement space to the ima...,Define AA:= Since a complement space to the im...
1,In van den Bergh (2009) it was speculated that...,In van den Bereh (2009) it was speculated that...,0907_0907.3715d_0907.3715_psfilefixedRTM_hocr,page5,In van den Bergh (2009) it was speculated that...,In van den Bereh (2009) it was speculated that...
2,noise due to the low number density of galaxie...,noise due to the low number density of galaxie...,1003_1003.4211d_boosting_psfilefixedRTM_hocr,page11,noise due to the low number density of galaxie...,noise due to the low number density of galaxie...
3,X-ray observations with the observatory have e...,X-ray observations with the observatory have e...,0709_0709.1336d_paper_rev3_aph_psfilefixedRTM_...,page0,X-ray observations with the observatory have e...,X-ray observations with the observatory have e...
4,"strong O band, however. Methane shows only one...",strong @ band. however. Methane shows only one...,0310_astro-ph0310805d_saumon_psfilefixedRTM_hocr,page3,"strong O band, however. Methane shows only one...",strong band. however. Methane shows only one ...
...,...,...,...,...,...,...
245,"motions one can estimate, e.g., the expected m...",motions one can estimate. e.g.. the expected m...,1011_1011.4816d_15641_psfilefixedRTM_hocr,page6,"motions one can estimate, e.g., the expected m...",motions one can estimate. e.g.. the expected m...
246,"numerous in our star sample (15381 stars), it ...",numerous in our star sample (15381 stars). it ...,0512_astro-ph0512352d_ngc7789_exp_psfilefixedR...,page7,"numerous in our star sample (15381 stars), it ...",numerous in our star sample (15381 stars). it ...
247,we summarize our main conclusions. We assume t...,we summarize our main conclusions. We assume t...,1008_1008.2986d_nsc_psfilefixedRTM_hocr,page2,we summarize our main conclusions. We assume t...,we summarize our main conclusions. We assume t...
248,RVM sign convention problem The phase of maxim...,RVM sign convention problem The phase of maxim...,1111_1111.4270d_FCamiloJ2030+3641_psfilefixedR...,page4,RVM sign convention problem The phase of maxim...,RVM sign convention problem The phase of maxim...


In [36]:
rerun_vocab = False

if rerun_vocab:
    start_time = time.time()
    vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
    print(len(vocabulary))
    with open(output_folder+"data/vocabulary_new_pages_new"+ender+".pkl", "wb") as file:
        pickle.dump(vocabulary, file)
    end_time = time.time()
    print('time delta =', end_time-start_time, 'seconds, or', 
          (end_time-start_time)/60., 'minutes, or',
         (end_time-start_time)/60./60., 'hours')

408
time delta = 1.479722023010254 seconds, or 0.024662033716837565 minutes, or 0.0004110338952806261 hours


In [37]:
def mask_rename(datain):
    mask = (pd.isnull(datain['gt'])) | (pd.isnull(datain['ocr']))
    # datain = datain.rename(columns={'words source':"ocr_to_input", 
    #                             'words source aligned': "ocr_aligned",
    #                             'words target aligned':"gs_aligned"})
    datain = datain.rename(columns={'ocr_clean':"ocr_to_input", 
                            'ocr': "ocr_aligned",
                            'gt':"gs_aligned"})
    return datain

In [38]:
# train_data.to_pickle(output_folder+"/data/train_new_pages"+ender+".pkl")
# train_data.shape

Format and save training data:

In [39]:
train_data = mask_rename(train_data)

In [40]:
start_time = time.time()
with Pool(4) as p:
    train_aligned = list(p.imap_unordered(create_windows, 
                                          tqdm(zip(train_data.ocr_aligned, 
                                                   train_data.gs_aligned, 
                                                   [window_length for x in train_data.ocr_aligned]), 
                                               total = len(train_data.ocr_aligned)),
                                          chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()
end_time = time.time()
print('total time =', end_time-start_time, 'seconds, or', (end_time-start_time)/60., 'minutes')

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

(3041135, 2)
total time = 7.2342798709869385 seconds, or 0.12057133118311564 minutes


In [41]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,In this section we discuss the effects of clus...,In this section we discuss the effects of clus...
1,n this section we discuss the effects of clust...,n this section we discuss the effects of clust...
2,this section we discuss the effects of cluste...,this section we discuss the effects of cluste...
3,this section we discuss the effects of cluster...,this section we discuss the effects of cluster...
4,his section we discuss the effects of cluster ...,his section we discuss the effects of cluster ...


In [42]:
train_aligned.to_pickle(output_folder+"data/train_aligned_new_pages"+ender+".pkl")

For dev:

In [43]:
val_data = mask_rename(val_data)

In [44]:
dev_aligned = val_data.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(766032, 2)


,source,target
0,total contribution of these two populations re...,total contribution of these two populations re...
1,otal contribution of these two populations res...,otal contribution of these two populations res...
2,tal contribution of these two populations resu...,tal contribution of these two populations resu...
3,al contribution of these two populations resul...,al contribution of these two populations resul...
4,l contribution of these two populations result...,l contribution of these two populations result...


In [45]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,total contribution of these two populations re...,total contribution of these two populations re...
1,otal contribution of these two populations res...,otal contribution of these two populations res...
2,tal contribution of these two populations resu...,tal contribution of these two populations resu...
3,al contribution of these two populations resul...,al contribution of these two populations resul...
4,l contribution of these two populations result...,l contribution of these two populations result...


In [46]:
dev_aligned.to_pickle(output_folder+"data/dev_aligned_new_pages"+ender+".pkl")

In [ ]:
test_data.to_csv(output_folder + "data/test_file"+ender+'csv',index=False)